In [1]:
from buildstock_query import BuildStockQuery
import pandas as pd

In [2]:
my_run = BuildStockQuery(db_name='euss-final',
                        table_name='euss_res_final_2018_550k_20220901',
                        workgroup='rescore',
                        buildstock_type='resstock')

INFO:buildstock_query.query_core:Loading euss_res_final_2018_550k_20220901 ...
INFO:botocore.tokens:Loading cached SSO token for nrel-sso
INFO:buildstock_query.query_core:21 queries cache read from None.
INFO:buildstock_query.query_core:21 queries cache is updated.
INFO:buildstock_query.main:Getting Success counts...
INFO:buildstock_query.report_query:Checking integrity with ts_tables ...
INFO:buildstock_query.query_core:No new queries to save.


         fail  unapplicable  success     Sum  Applied %  no-chng  bad-chng  \
upgrade                                                                      
0        1084             0   548916  550000        0.0        0         0   
1           1         17092   531823  548916       96.9     6097       949   
10          0             0   548916  548916      100.0        1      5310   
2           2         12117   536797  548916       97.8     6527       879   
3           0           656   548260  548916       99.9      807     29954   
4           0           656   548260  548916       99.9        4     20705   
5         107         16653   532156  548916       97.0      819     47818   
6           1          3584   545331  548916       99.3       31      2757   
7           2        148335   400579  548916       73.0     4169      7603   
8           3             0   548913  548916      100.0        2      6456   
9           0             0   548916  548916      100.0        1

In [3]:
baseline_agg = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=['geometry_building_type_recs'])
baseline_agg


,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu
0,Mobile Home,34491,8.351341e+06,1.399963e+06
1,Multi-Family with 2 - 4 Units,43868,1.062180e+07,1.215178e+06
2,Multi-Family with 5+ Units,100345,2.429664e+07,2.494200e+06
3,Single-Family Attached,32125,7.778459e+06,1.039893e+06
4,Single-Family Detached,338087,8.186135e+07,1.571373e+07


In [4]:
baseline_agg = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=['geometry_building_type_recs'],
                                       agg_func='max',
                                       get_nonzero_count=True)
baseline_agg

INFO:botocore.tokens:Loading cached SSO token for nrel-sso


,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu__max,fuel_use_electricity_total_m_btu__nonzero_units_count
0,Mobile Home,34491,8.351341e+06,635.684,8.351341e+06
1,Multi-Family with 2 - 4 Units,43868,1.062180e+07,535.279,1.062180e+07
2,Multi-Family with 5+ Units,100345,2.429664e+07,397.650,2.429664e+07
3,Single-Family Attached,32125,7.778459e+06,616.660,7.778459e+06
4,Single-Family Detached,338087,8.186135e+07,757.359,8.186135e+07


In [6]:
from sqlalchemy.sql import func as safunc
import sqlalchemy as sa
def my_non_zero_count(column):
    """
    Returns a SQL expression to count the number of non-zero values in a column.
    """
    return safunc.sum(sa.case([(column != 0, 1)], else_=0) * sa.column("build_existing_model.sample_weight"))

baseline_agg = my_run.agg.aggregate_annual(enduses=['fuel_use_natural_gas_total_m_btu'],
                                       group_by=['geometry_building_type_recs'],
                                       agg_func=my_non_zero_count,
                                       get_nonzero_count=True)
baseline_agg

,geometry_building_type_recs,sample_count,units_count,fuel_use_natural_gas_total_m_btu__my_non_zero_count,fuel_use_natural_gas_total_m_btu__nonzero_units_count
0,Mobile Home,34491,8.351341e+06,3.045282e+06,3.045282e+06
1,Multi-Family with 2 - 4 Units,43868,1.062180e+07,7.284269e+06,7.284269e+06
2,Multi-Family with 5+ Units,100345,2.429664e+07,1.338089e+07,1.338089e+07
3,Single-Family Attached,32125,7.778459e+06,5.766592e+06,5.766592e+06
4,Single-Family Detached,338087,8.186135e+07,5.395333e+07,5.395333e+07


In [8]:
baseline_agg = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=['geometry_building_type_recs'],
                                       get_nonzero_count=True)
baseline_agg

,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu,fuel_use_electricity_total_m_btu__nonzero_units_count
0,Mobile Home,34491,8.351341e+06,1.399963e+06,8.351341e+06
1,Multi-Family with 2 - 4 Units,43868,1.062180e+07,1.215178e+06,1.062180e+07
2,Multi-Family with 5+ Units,100345,2.429664e+07,2.494200e+06,2.429664e+07
3,Single-Family Attached,32125,7.778459e+06,1.039893e+06,7.778459e+06
4,Single-Family Detached,338087,8.186135e+07,1.571373e+07,8.186135e+07


In [20]:
my_run.save_cache()

INFO:buildstock_query.query_core:21 queries cache saved to .bsq_cache/euss_res_final_2018_550k_20220901_query_cache.pkl


In [21]:
all_enduses = my_run.get_cols(my_run.ts_table)
all_enduses = [str(x) for x in all_enduses if 'end_use__' in str(x)]
all_enduses

['euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__ceiling_fan__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__clothes_dryer__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__clothes_washer__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__cooling__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__cooling_fans_pumps__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__dishwasher__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__freezer__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__heating__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__heating_fans_pumps__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__heating_heat_pump_backup__kwh',
 'euss_res_final_2018_550k_20220901_timeseries.end_use__electricity__hot_tub_heater__kwh',
 'euss_res

In [24]:
ts_agg = my_run.agg.aggregate_timeseries(enduses=['fuel_use__electricity__total__kwh'],
                                         restrict=[('build_existing_model.state', ['CO'])],
                                         timestamp_grouping_func='month',
                                         group_by=['geometry_building_type_recs', 'build_existing_model.state'],
                                         get_query_only=False)
print(ts_agg)

INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.


      geometry_building_type_recs state       time  sample_count  \
0                     Mobile Home    CO 2018-01-01           391   
1                     Mobile Home    CO 2018-02-01           391   
2                     Mobile Home    CO 2018-03-01           391   
3                     Mobile Home    CO 2018-04-01           391   
4                     Mobile Home    CO 2018-05-01           391   
5                     Mobile Home    CO 2018-06-01           391   
6                     Mobile Home    CO 2018-07-01           391   
7                     Mobile Home    CO 2018-08-01           391   
8                     Mobile Home    CO 2018-09-01           391   
9                     Mobile Home    CO 2018-10-01           391   
10                    Mobile Home    CO 2018-11-01           391   
11                    Mobile Home    CO 2018-12-01           391   
12  Multi-Family with 2 - 4 Units    CO 2018-01-01           468   
13  Multi-Family with 2 - 4 Units    CO 2018-02-

In [14]:
# peak electricity use for each building in AL
ts_agg = my_run.agg.aggregate_timeseries(enduses=['fuel_use__electricity__total__kwh'],
                                         restrict=[('build_existing_model.state', ['AL'])],
                                         collapse_ts=True,
                                         agg_func='max',
                                         group_by=[my_run.bs_bldgid_column],
                                         get_query_only=False)
print(ts_agg)

INFO:buildstock_query.aggregate_query:Aggregation done accross timestamps. Result no longer a timeseries.
INFO:buildstock_query.aggregate_query:Restricting query to Upgrade 0.


      building_id  sample_count  units_count  \
0             186             1   242.131013   
1             212             1   242.131013   
2             239             1   242.131013   
3             273             1   242.131013   
4             307             1   242.131013   
...           ...           ...          ...   
9100       549793             1   242.131013   
9101       549800             1   242.131013   
9102       549804             1   242.131013   
9103       549820             1   242.131013   
9104       549999             1   242.131013   

      fuel_use__electricity__total__kwh__max  
0                                      1.681  
1                                      3.629  
2                                      3.507  
3                                      3.975  
4                                      4.906  
...                                      ...  
9100                                   3.993  
9101                                   6.897  


In [16]:
my_run.savings.savings_shape(upgrade_id=2, enduses=['fuel_use__electricity__total__kwh'],
                             restrict=[('build_existing_model.state', ['AL'])],
                             annual_only=False)

,time,sample_count,units_count,fuel_use__electricity__total__kwh__baseline,fuel_use__electricity__total__kwh__savings
0,2018-01-01 00:15:00,9105,2.204603e+06,2.344104e+06,798942.753525
1,2018-01-01 00:30:00,9105,2.204603e+06,2.380520e+06,804806.924523
2,2018-01-01 00:45:00,9105,2.204603e+06,2.377988e+06,802971.087184
3,2018-01-01 01:00:00,9105,2.204603e+06,2.370643e+06,801110.794613
4,2018-01-01 01:15:00,9105,2.204603e+06,2.327645e+06,794221.925170
...,...,...,...,...,...
35035,2018-12-31 23:00:00,9105,2.204603e+06,5.392025e+05,35243.137427
35036,2018-12-31 23:15:00,9105,2.204603e+06,4.739228e+05,36440.475284
35037,2018-12-31 23:30:00,9105,2.204603e+06,4.598230e+05,39609.485979
35038,2018-12-31 23:45:00,9105,2.204603e+06,4.500661e+05,41160.092985


In [8]:
my_run.savings.savings_shape(upgrade_id=2, enduses=['fuel_use__electricity__total__kwh'],
                             group_by=['geometry_building_type_recs'],
                             annual_only=False)

,time,sample_count,units_count,fuel_use__electricity__total__kwh__baseline,fuel_use__electricity__total__kwh__savings,geometry_building_type_recs
0,2018-01-01 00:15:00,703,9.361996e+06,1.098386e+07,2.682538e+06,Mobile Home
1,2018-01-01 00:30:00,703,9.361996e+06,1.093122e+07,2.634476e+06,Mobile Home
2,2018-01-01 00:45:00,703,9.361996e+06,1.077442e+07,2.593965e+06,Mobile Home
3,2018-01-01 01:00:00,703,9.361996e+06,1.080007e+07,2.552429e+06,Mobile Home
4,2018-01-01 01:15:00,703,9.361996e+06,1.059866e+07,2.516619e+06,Mobile Home
...,...,...,...,...,...,...
175195,2018-12-31 23:00:00,7121,9.483182e+07,3.171454e+07,4.143382e+06,Single-Family Detached
175196,2018-12-31 23:15:00,7121,9.483182e+07,2.811727e+07,3.993204e+06,Single-Family Detached
175197,2018-12-31 23:30:00,7121,9.483182e+07,2.729993e+07,4.172387e+06,Single-Family Detached
175198,2018-12-31 23:45:00,7121,9.483182e+07,2.672484e+07,4.160202e+06,Single-Family Detached


In [9]:
my_run.savings.savings_shape(upgrade_id=2, enduses=['fuel_use__electricity__total__kwh'],
                             group_by=['geometry_building_type_recs'],
                             get_quartiles=True,
                             annual_only=False)

,time,sample_count,units_count,fuel_use__electricity__total__kwh__baseline,fuel_use__electricity__total__kwh__savings,fuel_use__electricity__total__kwh__savings__quartiles,geometry_building_type_recs
0,2018-01-01 00:15:00,703,9.361996e+06,1.098386e+07,2.682538e+06,"[-0.05399999999999999, 0.0, 0.0155899999999999...",Mobile Home
1,2018-01-01 00:30:00,703,9.361996e+06,1.093122e+07,2.634476e+06,"[-0.031, 0.0, 0.0158356401384083, 0.1711655303...",Mobile Home
2,2018-01-01 00:45:00,703,9.361996e+06,1.077442e+07,2.593965e+06,"[-0.03300000000000003, 0.0, 0.0154038281979458...",Mobile Home
3,2018-01-01 01:00:00,703,9.361996e+06,1.080007e+07,2.552429e+06,"[-0.027999999999999997, 0.0, 0.016376282051282...",Mobile Home
4,2018-01-01 01:15:00,703,9.361996e+06,1.059866e+07,2.516619e+06,"[-0.024999999999999994, 0.0, 0.015727272727272...",Mobile Home
...,...,...,...,...,...,...,...
175195,2018-12-31 23:00:00,7121,9.483182e+07,3.171454e+07,4.143382e+06,"[-0.473, -0.020270068826590562, 0.0, 0.0040731...",Single-Family Detached
175196,2018-12-31 23:15:00,7121,9.483182e+07,2.811727e+07,3.993204e+06,"[-0.489, -0.02042967741935484, 0.0, 0.00376620...",Single-Family Detached
175197,2018-12-31 23:30:00,7121,9.483182e+07,2.729993e+07,4.172387e+06,"[-0.518, -0.017058074866310155, 0.0, 0.0040970...",Single-Family Detached
175198,2018-12-31 23:45:00,7121,9.483182e+07,2.672484e+07,4.160202e+06,"[-0.587, -0.017366656552689926, 0.0, 0.0041328...",Single-Family Detached
